<a href="https://colab.research.google.com/github/rajeswarimuppidi/EVA-4---Assignments---S6/blob/master/Session_6_All.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1)
        self.bn1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        self.bn2 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2)
        self.conv3 = nn.Conv2d(16, 10, 3, padding=1)
        self.bn3 = nn.BatchNorm2d(10)
        self.conv4 = nn.Conv2d(10, 10, 3, padding=1)
        self.bn4 = nn.BatchNorm2d(10)
        self.pool2 = nn.MaxPool2d(2, 2)
        self.conv5 = nn.Conv2d(10, 10, 3)
        self.bn5 = nn.BatchNorm2d(10)
        self.conv6 = nn.Conv2d(10, 10, 3)
        self.bn6 = nn.BatchNorm2d(10)
        self.gap = nn.AdaptiveAvgPool2d((1,1))
        self.conv7 = nn.Conv2d(10, 10, 1)
        self.dout = nn.Dropout(p=0.06)
           
    def forward(self, x):
        x = self.bn1(F.relu(self.conv1(x)))
        x = self.dout(x)
        x = self.pool1(self.bn2(F.relu(self.conv2(x))))
        x = self.dout(x)
        x = self.bn3(F.relu(self.conv3(x)))
        x = self.dout(x)
        x = self.pool2(self.bn4(F.relu(self.conv4(x))))
        x = self.dout(x)
        x = self.bn5(F.relu(self.conv5(x)))
        x = self.dout(x)
        x = self.bn6(F.relu(self.conv6(x)))
        x = self.gap(x)
        x = self.conv7(x)
        x = x.view(-1, 10)
        return F.log_softmax(x, dim=-1)

In [0]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
           Dropout-3           [-1, 16, 28, 28]               0
            Conv2d-4           [-1, 16, 28, 28]           2,320
       BatchNorm2d-5           [-1, 16, 28, 28]              32
         MaxPool2d-6           [-1, 16, 14, 14]               0
           Dropout-7           [-1, 16, 14, 14]               0
            Conv2d-8           [-1, 10, 14, 14]           1,450
       BatchNorm2d-9           [-1, 10, 14, 14]              20
          Dropout-10           [-1, 10, 14, 14]               0
           Conv2d-11           [-1, 10, 14, 14]             910
      BatchNorm2d-12           [-1, 10, 14, 14]              20
        MaxPool2d-13             [-1, 10, 7, 7]               0
          Dropout-14             [-1, 1

In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)

test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)



Extracting ../data/MNIST/raw/train-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/train-labels-idx1-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-images-idx3-ubyte.gz to ../data/MNIST/raw



Extracting ../data/MNIST/raw/t10k-labels-idx1-ubyte.gz to ../data/MNIST/raw
Processing...
Done!


In [0]:
from tqdm import tqdm

train_losses = []
test_losses = []
train_acc = []
test_acc = []

def train(model, device, train_loader, optimizer, epoch):
  model.train()
  pbar = tqdm(train_loader)
  correct = 0
  processed = 0
  for batch_idx, (data, target) in enumerate(pbar):
    # get samples
    data, target = data.to(device), target.to(device)

    # Init
    optimizer.zero_grad()
    # In PyTorch, we need to set the gradients to zero before starting to do backpropragation because PyTorch accumulates the gradients on subsequent backward passes. 
    # Because of this, when you start your training loop, ideally you should zero out the gradients so that you do the parameter update correctly.

    # Predict
    y_pred = model(data)

    # Calculate loss
    loss = F.nll_loss(y_pred, target)
    train_losses.append(loss)

    # Backpropagation
    loss.backward()
    optimizer.step()

    # Update pbar-tqdm
    
    pred = y_pred.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
    correct += pred.eq(target.view_as(pred)).sum().item()
    processed += len(data)

    pbar.set_description(desc= f'Loss={loss.item()} Batch_id={batch_idx} Train_Accuracy={100*correct/processed:0.2f}')
    train_acc.append(100*correct/processed)

def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)
    test_losses.append(test_loss)
    print('\nEpoch:',epoch)
    print('\nTest set: Average loss: {:.4f}, Test_Accuracy: {}/{} ({:.2f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))
    
    test_acc.append(100. * correct / len(test_loader.dataset))

In [0]:
#from torch.optim.lr_scheduler import StepLR

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
#scheduler = StepLR(optimizer, step_size=2, gamma=0.6)

for epoch in range(1, 41):
    train(model, device, train_loader, optimizer, epoch)
 #   if (epoch > 5):
  #   scheduler.step()
    test(model, device, test_loader)

Loss=0.21858690679073334 Batch_id=468 Train_Accuracy=88.60: 100%|██████████| 469/469 [00:12<00:00, 36.33it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 1

Test set: Average loss: 0.0799, Test_Accuracy: 9776/10000 (97.76%)



Loss=0.048823799937963486 Batch_id=468 Train_Accuracy=97.65: 100%|██████████| 469/469 [00:12<00:00, 36.34it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 2

Test set: Average loss: 0.0546, Test_Accuracy: 9847/10000 (98.47%)



Loss=0.11470664292573929 Batch_id=468 Train_Accuracy=98.17: 100%|██████████| 469/469 [00:13<00:00, 35.84it/s]
  0%|          | 0/469 [00:00<?, ?it/s]

...


Epoch: 19

Test set: Average loss: 0.0225, Test_Accuracy: 9924/10000 (99.24%)



Loss=0.010623052716255188 Batch_id=468 Train_Accuracy=99.20: 100%|██████████| 469/469 [00:12<00:00, 36.18it/s]
  0%|          | 0/469 [00:00<?, ?it/s]


Epoch: 20

Test set: Average loss: 0.0235, Test_Accuracy: 9931/10000 (99.31%)



Loss=0.04156263545155525 Batch_id=360 Train_Accuracy=99.19:  76%|███████▋  | 358/469 [00:10<00:03, 35.86it/s]

KeyboardInterrupt: ignored

Target:

As the parameters are reduced add dropout and GAP to the model.

Results:

Parameters: 9K

Best Training Accuracy: 99.43%

Best Test Accuracy: 99.51% (>99.4 seen consistently in last few epochs)

Analysis:

Model is light weight.

This is a fit model. 

Model can be further improved.


